In [43]:
# import gzip
import json

# Dataset: Clothing Fit Data (https://cseweb.ucsd.edu/~jmcauley/datasets.html#amazon_reviews)
# 1. Modcloth (download link: https://datarepo.eng.ucsd.edu/mcauley_group/data/modcloth/modcloth_final_data.json.gz)
# 2. RenttheRunway (download link: https://datarepo.eng.ucsd.edu/mcauley_group/data/renttherunway/renttherunway_final_data.json.gz)

# modcloth_file = gzip.open('/content/modcloth_final_data.json.gz') # this dataset does not have reviews
# modcloth_data = []
# for mc_item in modcloth_file:
#     modcloth_data.append(json.loads(mc_item))

# renttherunway_file = gzip.open('/kaggle/input/renttherunway/renttherunway_final_data.json')

renttherunway_file = open('/kaggle/input/renttherunway/renttherunway_final_data.json')

renttherunway_data = []
for rtrw_item in renttherunway_file:
  renttherunway_data.append(json.loads(rtrw_item))


In [44]:
print("renttherunway dataset size:", len(renttherunway_data))
# renttherunway_data = renttherunway_data[0:40000]

renttherunway dataset size: 192544


In [45]:
renttherunway_data = renttherunway_data[0:2000]

In [46]:
import torch

# Confirm that the GPU is detected
# torch.cuda.init()
# assert torch.cuda.is_available()

# Get the GPU device name.
# device_name = torch.cuda.get_device_name()
# n_gpu = torch.cuda.device_count()
# print(f"Found device: {device_name}, n_gpu: {n_gpu}")
# device = torch.device("cuda")

In [47]:
# Preprocess: extract only reviews and ratings, and remove null data
reviews = []
ratings = []
for item in renttherunway_data:
  if item['rating'] is not None and item['review_text'] is not None:
    reviews.append(item['review_text'])
    ratings.append(int(item['rating']))

print("dataset size after cleaning:", len(reviews))
print("Mean of all ratings: ", round(sum(ratings) / len(ratings), 2))


dataset size after cleaning: 2000
Mean of all ratings:  9.05


In [48]:
# # Data cleaning
# import re
import pandas as pd

df = pd.DataFrame({'Review': reviews, 'Rating': ratings})

In [49]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [50]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1600 200 200


In [51]:
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# example_text = 'I will watch Memento tonight. It is my favorite movie. I want to watch it again.'
# bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
#                        truncation=True, return_tensors="pt")


# print(bert_input['input_ids'])
# print(bert_input['token_type_ids'])
# print(bert_input['attention_mask'])

In [52]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# labels = {'2':0,
#           '4':1,
#           '6':2,
#           '8':3,
#           '10':4
#           }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [np.float32(label) for label in df['Rating']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['Review']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [53]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        

        return final_layer

In [54]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

#     criterion = nn.CrossEntropyLoss()
    criterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr = learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask).squeeze()
#                 print("output:",output)
#                 print("train_label:", train_label)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask).squeeze()

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f}')
                  

In [ ]:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

EPOCHS = 5
model = BertClassifier()
LR = 1e-5

import time
start = time.time()
              
train(model, df_train, df_val, LR, EPOCHS)

end = time.time()
print("Elapsed Time: {:.2f} seconds".format(end - start))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 800/800 [1:13:41<00:00,  5.53s/it]


Epochs: 1 | Train Loss:  3.945 | Val Loss:  1.118


100%|██████████| 800/800 [1:12:59<00:00,  5.47s/it]


Epochs: 2 | Train Loss:  0.823 | Val Loss:  1.003


100%|██████████| 800/800 [1:12:53<00:00,  5.47s/it]


Epochs: 3 | Train Loss:  0.550 | Val Loss:  0.979


100%|██████████| 800/800 [1:13:18<00:00,  5.50s/it]


Epochs: 4 | Train Loss:  0.390 | Val Loss:  0.937


100%|██████████| 800/800 [1:13:01<00:00,  5.48s/it]


In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    criterion = nn.MSELoss()

    if use_cuda:

        model = model.cuda()

    total_loss_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask).squeeze()
            
              batch_loss = criterion(output, test_label)
              total_loss_test += batch_loss.item()
              
    
    print(f'Val Loss: {total_loss_test / len(test_data): .3f}')

In [57]:
evaluate(model, df_test)

Val Loss:  0.823
